# Análise de vendas de um e-commerce

por [Lucas Fiorani Diniz](https://www.linkedin.com/in/lcfdiniz/)

## Solução do problema

Nessa etapa, o problema será finalmente analisado e resolvido, de forma a responder as perguntas levantadas no objetivo desse trabalho. Foram essas:

- Quais são os produtos mais vendidos no e-commerce?
- Qual o preço médio das vendas?
- Qual estado ou região do Brasil realiza mais compras online?
- Qual a forma de pagamento de preferência?
- Cada pedido possui em média quantos itens?

### 1. Quais são os produtos mais vendidos no e-commerce?

In [0]:
select product_id, count(*) as sales_count, avg(price) as avg_price
from venda
group by product_id
order by sales_count desc
limit 5

In [0]:
select *
from produto
where product_id in (
    select product_id
    from venda
    group by product_id
    order by count(*) desc
    limit 5
)

O produto mais vendido está presente em 467 transações, custa em média 92,13 U.M. e pertence à categoria "Cama, Mesa e Banho". Os produtos mais vendidos são apresentados na tabela abaixo:

| Produto | Quantidade | Categoria | Preço Médio|
|----------|----------|----------|----------|
| 1 | 467| Cama, Mesa e Banho | 92,13 |
| 2 | 431 | Móveis e Decoração | 87,26 |
| 3 | 352 | Ferramentas e Jardim | 75,50|
| 4 | 323 | Informática e Acessórios | 146,17 |
| 5 | 311 | Ferramentas e Jardim | 68,94 |

Ainda podemos descobrir qual o produto mais vendido em valor total de vendas:

In [0]:
select product_id, sum(price) as sum_price, avg(price) as avg_price
from venda
group by product_id
order by sum_price desc
limit 5

In [0]:
select *
from produto
where product_id in (
    select product_id
    from venda
    group by product_id
    order by sum(price) desc
    limit 5
)

O produto com maior arrecadação em vendas custa em média 341,63 U.M. e pertence à categoria "Beleza e Saúde". O produto mais vendido em quantidade é apenas o quinto com maior arrecadação em vendas.

### 2. Qual o preço médio das vendas?

In [0]:
select avg(price)
from venda

O preço média das vendas foi de 132,70 U.M. Sendo assim, o produto mais vendido em quantidade pode ser considerado barato em relação à média, enquanto o produto mais vendido em arrecadação total pode ser considerado caro.

### 3. Qual estado ou região do Brasil realiza mais compras online?

In [0]:
select customer_state, count(*) as orders_count
from pedido as p 
    left join cliente as c 
    on p.customer_id = c.customer_id
group by customer_state
order by count(*) desc
limit 5

As regiões **Sudeste** e Sul do Brasil são as que mais realizam compras, dominando os cinco estados que mais fizeram pedidos. O estado de **São Paulo** é o mais ativo, com um número total de pedidos mais que três vezes superior ao do segundo colocado, Rio de Janeiro.

| Estado | Número de Pedidos|
|----------|----------|
| São Paulo | 41746|
| Rio de Janeiro | 12852|
| Minas Gerais | 11635|
| Rio Grande do Sul | 5466|
| Paraná | 5045|

### 4. Qual a forma de pagamento de preferência?

In [0]:
select payment_type, count(*) as count_payments
from pagamento
group by payment_type
order by count_payments desc

O **cartão de crédito** é a forma de pagamento mais popular (76795 transações), seguido pelo boleto (19784 transações) e voucher (5775 transações).

### 5. Cada pedido possui em média quantos itens?

In [0]:
select avg(order_count) as avg_order_count
from (
    select order_id, count(*) as order_count
    from venda
    group by order_id
)

Os pedidos possuem, em média, um único item.

### Explorando as regras de associação

Será verificada a coocorrência de itens nas transações realizadas, a partir de regras de associação. A capacidade de identificar as associações de itens mais relevantes permitiria ao e-commerce recomendar produtos em carrinho de compras, por exemplo.

In [0]:
select order_id, 
    count(*) as order_count, 
    listagg(product_id, ', ') within group (order by product_id) as product_list
from venda
group by order_id
having order_count > 1

In [0]:
select product_list, count(*) as count_product_list
from (
    select order_id, 
        count(*) as order_count, 
        listagg(product_id, ', ') within group (order by product_id) as product_list
    from venda
    group by order_id
    having order_count > 1
)
group by product_list
order by count_product_list desc

Para um total de 99441 pedidos, as coocorrências mais comuns ocorrem em até 34 pedidos, o que indica um valor de suporte extremamente baixo. As três coocorrências mais frequentes encontram-se listadas abaixo, acompanhadas de seu valor de suporte:

| Produto 1 | Produto 2 | Coocorrências | Suporte |
|----------|----------|----------|----------|
| 36f60d45225e60c7da4558b070ce4b60 | e53e557d5a159f5aa2c5e995dfdf244b |	34 | 3.42e-04 |
| 35afc973633aaeb6b877ff57b2793310 | 99a4788cb24856965c36a24e339b6058 | 26 | 2.61e-04 |
| 4fcb3d9a5f4871e8362dfedbdb02b064 | f4f67ccaece962d013a4e1d7dc3a61f7 | 17 | 1.71e-04 |

Agora analisaremos a confiança dessas associações:

In [0]:
select product_id, count(*) as sales_count
from venda
where product_id in (
    select split_part(product_list, ', ', 1)
    from (
        select order_id, 
            count(*) as order_count, 
            listagg(product_id, ', ') within group (order by product_id) as product_list
        from venda
        group by order_id
        having order_count > 1
    )
    group by product_list
    order by count(*) desc
    limit 3
) or product_id in (
    select split_part(product_list, ', ', 2)
    from (
        select order_id, 
            count(*) as order_count, 
            listagg(product_id, ', ') within group (order by product_id) as product_list
        from venda
        group by order_id
        having order_count > 1
    )
    group by product_list
    order by count(*) desc
    limit 3
)
group by product_id
order by sales_count desc

| Antecedente (X) | Consequente (Y) | Coocorrências | Ocorrências de X | Confiança |
|----------|----------|----------|----------|----------|
| 36f60d45225e60c7da4558b070ce4b60 | e53e557d5a159f5aa2c5e995dfdf244b |	34 | 111 | 0,31 |
| 35afc973633aaeb6b877ff57b2793310 | 99a4788cb24856965c36a24e339b6058 | 26 | 156 | 0,17 |
| f4f67ccaece962d013a4e1d7dc3a61f7 | 4fcb3d9a5f4871e8362dfedbdb02b064 | 17 | 56 | 0,30 |

Por fim, iremos analisar o Lift, ou seja, o aumento na probabilidade de termos Y no carrinho dado que X está no carrinho.

| Antecedente (X) | Consequente (Y) | % Ocorrências de Y | Confiança | Lift |
|----------|----------|----------|----------|----------|
| 36f60d45225e60c7da4558b070ce4b60 | e53e557d5a159f5aa2c5e995dfdf244b |	1,57e-03 | 0,31 | 197,61 |
| 35afc973633aaeb6b877ff57b2793310 | 99a4788cb24856965c36a24e339b6058 | 4,70e-03 | 0,17 | 36,20 |
| f4f67ccaece962d013a4e1d7dc3a61f7 | 4fcb3d9a5f4871e8362dfedbdb02b064 | 8,95e-04 | 0,30 | 335,19 |

A regra de associação mais forte parece ser `f4f67ccaece962d013a4e1d7dc3a61f7` -> `4fcb3d9a5f4871e8362dfedbdb02b064`.

In [0]:
select *
from produto
where product_id in (
    'f4f67ccaece962d013a4e1d7dc3a61f7',
    '4fcb3d9a5f4871e8362dfedbdb02b064'
)

É interessante notar que ambos os produtos são da categoria automotiva, o que reforça a forte associação observada entre esses itens.